In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co

import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import matplotlib.pyplot as mplt



from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
import requests
from io import BytesIO

In [71]:
AS= pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\All_Streaming_Shows.csv',sep=',')
AS.shape[0] #12 353
AS.head(5)

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video


In [3]:
AS.dtypes

Series Title           object
Year Released           int64
Content Rating         object
IMDB Rating           float64
R Rating                int64
Genre                  object
Description            object
No of Seasons          object
Streaming Platform     object
dtype: object

Plein de notes IMDB !!!

In [4]:
AS['IMDB Rating'].notna().sum()

10207

Si on filtre netflix

In [5]:
AS['Streaming Platform'].isna().sum() #1983 valeurs manquantes
AS['Streaming Platform'].isin(['Netflix']).sum() #1 427 valeurs pour Netflix

1427

### Création du AS_N ac seulement Netflix

In [6]:
#AS_N=AS[AS['Streaming Platform'].isin(['Netflix'])]
#gfg_csv_data = AS_N.to_csv('AS_N.csv', index = True) 

In [78]:
AS_N= pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\AS_N.csv',sep=',')
del AS_N['Unnamed: 0']

In [79]:
AS_N.head(5)

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
2,Dark,2017,16+,8.8,95,"Crime,Drama",A missing child causes four families to help e...,3Seasons,Netflix
3,Sherlock,2010,16+,9.1,94,"Action & Adventure,Crime",A modern update finds the famous sleuth and hi...,4Seasons,Netflix
4,Better Call Saul,2015,18+,8.7,92,"Comedy,Crime",Six years before Saul Goodman meets Walter Whi...,5Seasons,Netflix


# Stat des

Baisse en 2020 -> année tronquée + corona

In [80]:
#creating the plot of number of movies released each year and the cumsum of that calculation
moviesReleasedEachYear = AS_N['Year Released'].value_counts().sort_index()
cumsumMoviesReleased = AS_N['Year Released'].value_counts().sort_index().cumsum()

trace1 = go.Bar(x = list(moviesReleasedEachYear.index), y = list(moviesReleasedEachYear.values),name = 'Movies Released Each Year')
trace2 = go.Scatter(x = list(cumsumMoviesReleased.index), y = list(cumsumMoviesReleased.values), mode='lines',name = 'Trendline of Number of Movies')

fig = plt.subplots.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

fig.update_layout(template = 'plotly_white',margin=dict(l=80, r=80, t=25, b=10),
                  title = { 'text' : '<b>Web Series over the Years</b>', 'x' : 0.5},
                 font_family = 'Calibri',title_font_color= 'black', showlegend = True)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

genres les mieux notés

In [81]:
genres = ", ".join(AS_N['Genre']).split(", ")
genres = ",".join(AS_N['Genre']).split(",")
for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008']:
    while i in genres:
        genres.remove(i)


In [64]:
mean_imdb_rating = {}
mean_r_rating = {}

colors_imdb = ['lightslategray'] * len(genres)
colors_r = ['lightslategray'] * len(genres)

for i,genre in enumerate(genres):
    #calculating the mean imdb and rotten tomatoes rating for each genre
    mean_imdb_rating[genre] = AS_N[(AS_N[genre] == 1 )& (AS_N['IMDB Rating'] != -1)]['IMDB Rating'].mean()
    mean_r_rating[genre] = AS_N[(AS_N[genre] == 1 )& (AS_N['R Rating'] != -1)]['R Rating'].mean()

KeyError: 'Crime'

In [63]:
    
    #if the mean rating for that genre is more than the overall rating, I change the color of the bar to rosybrown to make it stand out
    if(mean_imdb_rating[genre] > overall_mean_imdb):
        colors_imdb[i] = 'rosybrown'
    if(mean_r_rating[genre] > overall_mean_r):
        colors_r[i] = 'rosybrown'
#creating subplots to plot for both IMDB ratings as well as Rotten Tomatoes rating. 
fig = plt.subplots.make_subplots(rows = 1, cols = 2,
                                 horizontal_spacing=0.15, 
                                 subplot_titles=['IMDB Ratings','Rotten Tomatoes Rating'])

#adding graph for IMDB rating for each genre
trace_imdb = go.Bar(x = list(mean_imdb_rating.values()), y = list(mean_imdb_rating.keys()), 
                    name = 'Mean IMDB Rating', orientation = 'h',
                    marker_color = colors_imdb)
#adding the mean imdb rating to the graph so that we compare individual genres to this threshold
trace_mean_imdb = go.Scatter(y = list(mean_imdb_rating.keys()), x = [overall_mean_imdb]*len(mean_imdb_rating),
                             name = 'Overall Mean IMDB Rating',
                             mode = 'lines', line = {'color' : '#ffb6c1'})

#adding graph for Rotten Tomatoes rating for each genre
trace_r = go.Bar(x = list(mean_r_rating.values()), y = list(mean_r_rating.keys()),
                 name = "Mean Rotten Tomattoes Rating", orientation = 'h', 
                 marker_color = colors_r)
#adding the mean rotten tomatoes rating to the graph so that we compare individual genres to this threshold
trace_mean_r = go.Scatter(y = list(mean_r_rating.keys()), x = [overall_mean_r]*len(mean_r_rating), 
                          name = 'Overall Mean Rotten Tomatoes Rating',
                          mode = 'lines', line = {'color' : '#ffb6c1'})

#adding each trace to the their respective subplots
fig.add_trace(trace_imdb, row = 1, col = 1)
fig.add_trace(trace_mean_imdb, row = 1, col = 1)
fig.add_trace(trace_r, row = 1, col = 2)
fig.add_trace(trace_mean_r, row = 1, col = 2)

fig.update_layout(template = 'plotly_white',
                  height = 800, margin=dict(l=80, r=80, t=50, b=20),
                  title = { 'text' : '<b>Ratings in Each Genre</b>', 'x' : 0.5},
                  font_family = 'Fira Code',title_font_color= 'crimson', 
                  showlegend = False)
fig.show()

NameError: name 'selected_genres' is not defined

### Travail sur le genre

Contenu ajouté sur Netflix overall en fct du genre

In [83]:
def count_genra (data_f,nombre=22,date="Netflix"):
    
    """on récupère les catégories"""
    col = "Genre"
    categories = ", ".join(data_f['Genre']).split(", ")
    #print (categories)
    categories = ",".join(data_f['Genre']).split(",")
    #print (categories)
    
    """on enlève les catégories débiles"""
    for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008']:
        while i in categories:
            categories.remove(i)

    #print(categories)

    """On compte le nombre d'occurences de chaque type et on garde les 50 qui apparaissent le plus souvent"""
    counter_list = co.Counter(categories).most_common(nombre)
    #print (type(counter_list_all))

    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [_[0] for _ in counter_list][::-1]
    #print (labels)

    """Idem pour le nombre d'occurrences"""
    values = [_[1] for _ in counter_list][::-1]

    trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="#a678de"))

    data = [trace1]
    layout = go.Layout(title="Content added on "+str(date), legend=dict(x=0.1, y=1.1, orientation="h"))
    fig = go.Figure(data, layout=layout)
    fig.show()

    return(trace1)

In [84]:
count_genra(AS_N,40)

Bar({
    'marker': {'color': '#a678de'},
    'name': 'TV Shows',
    'orientation': 'h',
    'x': [3, 4, 5, 5, 5, 15, 20, 21, 22, 22, 25, 26, 34, 38, 40, 50, 54, 57, 71,
          103, 104, 192, 206, 212, 216, 359, 577],
    'y': [Travel, Stand-up & Talk, Home & Garden, Musical, Cult, LGBTQ, Sport,
          Food, Game Show, Biography, History, Science-Fiction, Horror, Anime,
          Mystery, Fantasy, Children, Thriller, Family, Romance, Reality,
          Animation, Crime, Action & Adventure, Documentary, Comedy, Drama]
})

On crée une liste de toutes les dates ou une série à été released.

In [85]:
liste_date=[]
for i in range(1969,2021):
    if AS_N['Year Released'].isin([i]).sum() != 0:
        liste_date.append(i)
print (liste_date)

[1969, 1972, 1983, 1986, 1987, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [86]:
def count_genra_y (annee,nombre=22):
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    """on récupère les catégories"""
    col = "Genre"
    categories = ", ".join(data_f['Genre']).split(", ")
    #print (categories)
    categories = ",".join(data_f['Genre']).split(",")
    #print (categories)
    
    """on enlève les catégories débiles"""
    for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008']:
        while i in categories:
            categories.remove(i)

    #print(categories)

    """On compte le nombre d'occurences de chaque type et on garde les 50 qui apparaissent le plus souvent"""
    counter_list = co.Counter(categories).most_common(nombre)
    #print (type(counter_list_all))

    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [_[0] for _ in counter_list][::-1]
    #print (labels)

    """Idem pour le nombre d'occurrences"""
    values = [_[1] for _ in counter_list][::-1]

    trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color='#a678de'))

    data = [trace1]
    layout = go.Layout(title="Content added on "+str(annee), legend=dict(x=0.1, y=1.1, orientation="h"))
    fig = go.Figure(data, layout=layout)
    fig.show()

    return(trace1)

In [87]:
count_genra_y(2018,10)

Bar({
    'marker': {'color': '#a678de'},
    'name': 'TV Shows',
    'orientation': 'h',
    'x': [14, 14, 16, 20, 39, 39, 44, 55, 62, 87],
    'y': [Family, Thriller, Romance, Reality, Animation, Action & Adventure,
          Crime, Documentary, Comedy, Drama]
})

In [88]:
count_genra_y(2017,5)

Bar({
    'marker': {'color': '#a678de'},
    'name': 'TV Shows',
    'orientation': 'h',
    'x': [27, 29, 36, 47, 80],
    'y': [Action & Adventure, Documentary, Crime, Comedy, Drama]
})

In [15]:
AS_2017=AS_N[AS_N['Year Released'].isin([2017])]
AS_2018=AS_N[AS_N['Year Released'].isin([2018])]

In [30]:
def prep (annee,nombre=10):
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    
    """on récupère les catégories"""
    col = "Genre"
    categories = ", ".join(data_f['Genre']).split(", ")
    #print (categories)
    categories = ",".join(data_f['Genre']).split(",")
    #print (categories)
    
    """on enlève les catégories débiles"""
    for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008']:
        while i in categories:
            categories.remove(i)

    #print(categories)

    """On compte le nombre d'occurences de chaque type et on garde les 50 qui apparaissent le plus souvent"""
    counter_list = co.Counter(categories).most_common(nombre)
    #print (type(counter_list_all))

    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [_[0] for _ in counter_list][::-1]
    #print (labels)

    """Idem pour le nombre d'occurrences"""
    values = [_[1] for _ in counter_list][::-1]
    return (values,labels)

In [31]:

    
    fig = go.Figure(data=[
        go.Bar(name=str(2018), y=prep(2018)[0], x=prep(2018)[1]),
        go.Bar(name='2020', y=prep(2020)[0], x=prep(2020)[1])
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    fig.show()
    

In [32]:
def genra_comp(year1,year2):
    
    fig = go.Figure(data=[
        go.Bar(name=str(year1), y=prep(year1)[0], x=prep(year1)[1]),
        go.Bar(name=str(year2), y=prep(year2)[0], x=prep(year2)[1])
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    fig.show()
    print ("vive la vie")
    return ("on a une figure")

In [33]:
genra_comp(2017,2018)

vive la vie


'on a une figure'

In [19]:
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()